In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.style.use('seaborn-pastel')

### 1. Load IAM

In [ ]:
fund_data= pd.read_csv("FUND/FUND_socio_scc_complete.csv", index_col=0)
page_data= pd.read_csv("PAGE/PAGE_socio_scc_complete.csv", index_col=0)
dice_data= pd.read_csv("DICE/DICE_socio_scc_complete.csv", index_col=0)

In [ ]:
fund_data_use= fund_data.loc[(fund_data["Year"] >= 2020) & (fund_data["Year"] <= 2100)].reset_index(drop=True)
fund_data_current= fund_data_use.copy()
fund_data_current.iloc[:, -13:]= fund_data_current.iloc[:, -13:] * 1.98

In [ ]:
fund_data_current

In [ ]:
page_data_current= page_data.copy()
page_data_current.iloc[:, -13:]= page_data_current.iloc[:, -13:] * 1.27
page_data_current["popu"]= page_data_current["popu"]/1000


In [ ]:
page_data_current.columns = ['Year', 'popu', 'GDP_percap', 'emiss_total', 'SCC', 'PAGE_1', 'PAGE_2', 'PAGE_3', 'PAGE_4', 
                                                                                 'PAGE_5', 'PAGE_6', 'PAGE_7', 'PAGE_8', 'PAGE_9',
                                                                                 'PAGE_10', 'PAGE_11', 'PAGE_12']

In [ ]:
page_data_current

In [ ]:
dice_data_current= dice_data.copy()

dice_data_current.iloc[:, -13:]= dice_data_current.iloc[:, -13:] * 1.5
dice_data_current.rename(columns={"emiss_tot": "emiss_total"}, inplace=True)
dice_data_current["popu"]= dice_data_current["popu"]/1000

dice_data_current


### Make X and y dataset

In [ ]:
PAGE_X_df= page_data_current.iloc[:, :4]
PAGE_y_df= page_data_current.iloc[:, -13:]

FUND_X_df= fund_data_current.iloc[:, :4]
FUND_y_df= fund_data_current.iloc[:, -13:]

DICE_X_df= dice_data_current.iloc[:, :4]
DICE_y_df= dice_data_current.iloc[:, -13:]



PAGE_y_df.columns= ['SCC', 'SCC_1', 'SCC_2', 'SCC_3', 'SCC_4', 'SCC_5', 'SCC_6', 
                            'SCC_7', 'SCC_8', 'SCC_9', 'SCC_10', 'SCC_11', 'SCC_12']

FUND_y_df.columns= ['SCC', 'SCC_1', 'SCC_2', 'SCC_3', 'SCC_4', 'SCC_5', 'SCC_6', 
                            'SCC_7', 'SCC_8', 'SCC_9', 'SCC_10', 'SCC_11', 'SCC_12']

DICE_y_df.columns= ['SCC', 'SCC_1', 'SCC_2', 'SCC_3', 'SCC_4', 'SCC_5', 'SCC_6', 
                            'SCC_7', 'SCC_8', 'SCC_9', 'SCC_10', 'SCC_11', 'SCC_12']




In [ ]:
X_df= pd.concat([FUND_X_df, PAGE_X_df, DICE_X_df]).reset_index(drop=True)
X_df

In [ ]:
y_df= pd.concat([FUND_y_df, PAGE_y_df, DICE_y_df]).reset_index(drop=True)
y_df

In [ ]:
y_df_prtp15= y_df[['SCC_1', 'SCC_2', 'SCC_3', 'SCC_4']]
y_df_prtp1= y_df[['SCC_5', 'SCC_6', 'SCC_7', 'SCC_8']]
y_df_prtp3= y_df[['SCC_9', 'SCC_10', 'SCC_11', 'SCC_12']]

### 2. Load the mcs input fund and page

In [ ]:
mcs_folder= "MCS_folder_IAM"

from glob import glob
fund_mcs_filename= glob(mcs_folder + "/" + "FUND*.csv")
page_mcs_filename= glob(mcs_folder + "/" + "PAGE*.csv")
dice_mcs_filename= glob(mcs_folder + "/" + "DICE*.csv")

len(fund_mcs_filename), len(page_mcs_filename), len(dice_mcs_filename)

In [ ]:
fund_mcs_data= []
for i in range(len(fund_mcs_filename)):
    fund_mcs_data.append(pd.read_csv(fund_mcs_filename[i], index_col=0))


page_mcs_data= []
for i in range(len(page_mcs_filename)):
    page_mcs_data.append(pd.read_csv(page_mcs_filename[i], index_col=0))


dice_mcs_data= []
for i in range(len(dice_mcs_filename)):
    dice_mcs_data.append(pd.read_csv(dice_mcs_filename[i], index_col=0))


In [ ]:
fund_mcs_data[0]

In [ ]:
page_mcs_data[0]

In [ ]:
dice_mcs_data[0]

In [ ]:
year_all= fund_data_current["Year"].copy() 

In [ ]:
year_all

### 2. Load SSP baseline MCS

In [ ]:
ssp_base_mcs_folder= "MCS_folder_ssp_baseline_all"

from glob import glob
ssp_base_mcs_filename= glob(ssp_base_mcs_folder + "/" + "*.csv")


len(ssp_base_mcs_filename)

In [ ]:
ssp_base_mcs_data= []
for i in range(len(ssp_base_mcs_filename)):
    ssp_base_mcs_data.append(pd.read_csv(ssp_base_mcs_filename[i], index_col=0))
    ssp_base_mcs_data[i].rename(columns= {"emiss_tot": "emiss_total"}, inplace=True)

In [ ]:
ssp_base_mcs_data[0]

In [ ]:
fund_mcs_data[0]

### 2. Load CCS MCS

In [ ]:
ccs_mcs_folder= "MCS_folder_ssp_ccs_all"

from glob import glob
ssp_ccs_mcs_filename= glob(ccs_mcs_folder + "/" + "*.csv")

len(ssp_ccs_mcs_filename)

##### exclude too extreme

In [ ]:
exclude_scen= ["SSP1-45",
               "SSP2-60",
               "SSP3-60",
               "SSP4-45",
               "SSP4-60",
               "SSP5-45",
               "SSP5-60"]

In [ ]:
# Get the indices of the filenames not excluded
remained_id = [i for i, f in enumerate(ssp_ccs_mcs_filename) if all(exclude not in f for exclude in exclude_scen)]

print(remained_id)

In [ ]:
len(remained_id)

In [ ]:
ssp_ccs_mcs_filename_filter= []

for i in remained_id:
    ssp_ccs_mcs_filename_filter.append(ssp_ccs_mcs_filename[i])

In [ ]:
len(ssp_ccs_mcs_filename_filter)

In [ ]:
ssp_ccs_mcs_data= []
for i in range(len(ssp_ccs_mcs_filename_filter)):
    ssp_ccs_mcs_data.append(pd.read_csv(ssp_ccs_mcs_filename_filter[i], index_col=0))

In [ ]:
ssp_ccs_mcs_data[0]

In [ ]:
for i in range(len(ssp_ccs_mcs_data)):
    ssp_ccs_mcs_data[i].rename(columns={"emiss_tot": "emiss_total"}, inplace=True)

In [ ]:
ssp_ccs_mcs_data[0]

### 3. Scaling mcs input

In [ ]:
def scaling_transform_mod(choose_scaler, for_scale, new_data=None, inverse=None, for_inverse=None):
    
    if choose_scaler == "minmax":
        scaler = MinMaxScaler()
        scaler.fit(for_scale)
        data_scaled = scaler.transform(new_data)
    else:
        scaler = StandardScaler()
        scaler.fit(for_scale)
        data_scaled = scaler.transform(new_data)


    if inverse == 'yes':
        scaler.fit(for_scale)
        data_inverse = scaler.inverse_transform(for_inverse)
        return data_inverse
    else:
        return data_scaled

In [ ]:
scaler= 'minmax'

fund_input_scld= []
for i in range(len(fund_mcs_data)):
    fund_input_scld.append(scaling_transform_mod(choose_scaler= scaler, 
                                                     for_scale= X_df.iloc[:, 1:],  
                                                     new_data= fund_mcs_data[i].iloc[:, 1:]))

page_input_scld= []
for i in range(len(page_mcs_data)):
    page_input_scld.append(scaling_transform_mod(choose_scaler= scaler, 
                                                     for_scale= X_df.iloc[:, 1:],  
                                                     new_data= page_mcs_data[i].iloc[:, 1:]))


dice_input_scld= []
for i in range(len(dice_mcs_data)):
    dice_input_scld.append(scaling_transform_mod(choose_scaler= scaler, 
                                                     for_scale= X_df.iloc[:, 1:],  
                                                     new_data= dice_mcs_data[i].iloc[:, 1:]))


ssp_base_input_scld= []
for i in range(len(ssp_base_mcs_data)):
    ssp_base_input_scld.append(scaling_transform_mod(choose_scaler= scaler, 
                                                     for_scale= X_df.iloc[:, 1:],  
                                                     new_data= ssp_base_mcs_data[i].iloc[:, 1:]))


ssp_ccs_input_scld= []
for i in range(len(ssp_ccs_mcs_data)):
    ssp_ccs_input_scld.append(scaling_transform_mod(choose_scaler= scaler, 
                                                     for_scale= X_df.iloc[:, 1:],  
                                                     new_data= ssp_ccs_mcs_data[i].iloc[:, 1:]))

### 4. Load trained model and prediction

In [ ]:
SVR_folder_prtp15= "SVR_01_1030_prtp_15"
SVR_folder_prtp1= "SVR_01_1030_prtp_1"
SVR_folder_prtp3= "SVR_01_1030_prtp_3"

In [ ]:
""" LOAD MODEL """
import joblib
model_prtp15= joblib.load(SVR_folder_prtp15 + "/" + "_shuffle_1030_3iams_prtp_15_SVR_model.pkl")
model_prtp1= joblib.load(SVR_folder_prtp1 + "/" + "_shuffle_1030_3iams_prtp_1_SVR_model.pkl")
model_prtp3= joblib.load(SVR_folder_prtp3 + "/" + "_shuffle_1030_3iams_prtp_3_SVR_model.pkl")

In [ ]:
fund_pred_prtp15=[]
fund_pred_prtp1=[]
fund_pred_prtp3=[]

for i in range(len(fund_input_scld)):
    fund_pred_prtp15.append(model_prtp15.predict(fund_input_scld[i]))
    fund_pred_prtp1.append(model_prtp1.predict(fund_input_scld[i]))
    fund_pred_prtp3.append(model_prtp3.predict(fund_input_scld[i]))

In [ ]:
page_pred_prtp15=[]
page_pred_prtp1=[]
page_pred_prtp3=[]

for i in range(len(page_input_scld)):
    page_pred_prtp15.append(model_prtp15.predict(page_input_scld[i]))
    page_pred_prtp1.append(model_prtp1.predict(page_input_scld[i]))
    page_pred_prtp3.append(model_prtp3.predict(page_input_scld[i]))

In [ ]:
dice_pred_prtp15=[]
dice_pred_prtp1=[]
dice_pred_prtp3=[]

for i in range(len(dice_input_scld)):
    dice_pred_prtp15.append(model_prtp15.predict(dice_input_scld[i]))
    dice_pred_prtp1.append(model_prtp1.predict(dice_input_scld[i]))
    dice_pred_prtp3.append(model_prtp3.predict(dice_input_scld[i]))

In [ ]:
ssp_base_pred_prtp15=[]
ssp_base_pred_prtp1=[]
ssp_base_pred_prtp3=[]

for i in range(len(ssp_base_input_scld)):
    ssp_base_pred_prtp15.append(model_prtp15.predict(ssp_base_input_scld[i]))
    ssp_base_pred_prtp1.append(model_prtp1.predict(ssp_base_input_scld[i]))
    ssp_base_pred_prtp3.append(model_prtp3.predict(ssp_base_input_scld[i]))

In [ ]:
ssp_ccs_pred_prtp15=[]
ssp_ccs_pred_prtp1=[]
ssp_ccs_pred_prtp3=[]

for i in range(len(ssp_ccs_input_scld)):
    ssp_ccs_pred_prtp15.append(model_prtp15.predict(ssp_ccs_input_scld[i]))
    ssp_ccs_pred_prtp1.append(model_prtp1.predict(ssp_ccs_input_scld[i]))
    ssp_ccs_pred_prtp3.append(model_prtp3.predict(ssp_ccs_input_scld[i]))

#### Inverse

In [ ]:

def scaling_transform(choose_scaler, for_scale, inverse=None, for_inverse=None):
    
    if choose_scaler == "minmax":
        scaler = MinMaxScaler()
        data_scaled = scaler.fit_transform(for_scale)
    else:
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(for_scale)


    if inverse == 'yes':
        data_inverse = scaler.inverse_transform(for_inverse)
        return data_inverse
    else:
        return data_scaled

In [ ]:
fund_pred_inv_prtp15= []
fund_pred_inv_prtp1= []
fund_pred_inv_prtp3= []

for i in range(len(fund_input_scld)):
    fund_pred_inv_prtp15.append(scaling_transform(choose_scaler= scaler,
                                                          for_scale= y_df_prtp15,
                                                          inverse='yes',
                                                          for_inverse= fund_pred_prtp15[i]))
    fund_pred_inv_prtp1.append(scaling_transform(choose_scaler= scaler,
                                                         for_scale= y_df_prtp1,
                                                         inverse='yes',
                                                         for_inverse= fund_pred_prtp1[i]))
    fund_pred_inv_prtp3.append(scaling_transform(choose_scaler= scaler,
                                                         for_scale= y_df_prtp3,
                                                         inverse='yes',
                                                         for_inverse= fund_pred_prtp3[i]))


In [ ]:
page_pred_inv_prtp15= []
page_pred_inv_prtp1= []
page_pred_inv_prtp3= []

for i in range(len(page_input_scld)):
    page_pred_inv_prtp15.append(scaling_transform(choose_scaler= scaler,
                                                          for_scale= y_df_prtp15,
                                                          inverse='yes',
                                                          for_inverse= page_pred_prtp15[i]))
    page_pred_inv_prtp1.append(scaling_transform(choose_scaler= scaler,
                                                         for_scale= y_df_prtp1,
                                                         inverse='yes',
                                                         for_inverse= page_pred_prtp1[i]))
    page_pred_inv_prtp3.append(scaling_transform(choose_scaler= scaler,
                                                         for_scale= y_df_prtp3,
                                                         inverse='yes',
                                                         for_inverse= page_pred_prtp3[i]))


In [ ]:
dice_pred_inv_prtp15= []
dice_pred_inv_prtp1= []
dice_pred_inv_prtp3= []

for i in range(len(dice_input_scld)):
    dice_pred_inv_prtp15.append(scaling_transform(choose_scaler= scaler,
                                                          for_scale= y_df_prtp15,
                                                          inverse='yes',
                                                          for_inverse= dice_pred_prtp15[i]))
    dice_pred_inv_prtp1.append(scaling_transform(choose_scaler= scaler,
                                                         for_scale= y_df_prtp1,
                                                         inverse='yes',
                                                         for_inverse= dice_pred_prtp1[i]))
    dice_pred_inv_prtp3.append(scaling_transform(choose_scaler= scaler,
                                                         for_scale= y_df_prtp3,
                                                         inverse='yes',
                                                         for_inverse= dice_pred_prtp3[i]))


In [ ]:
ssp_base_pred_inv_prtp15= []
ssp_base_pred_inv_prtp1= []
ssp_base_pred_inv_prtp3= []

for i in range(len(ssp_base_input_scld)):
    ssp_base_pred_inv_prtp15.append(scaling_transform(choose_scaler= scaler,
                                                          for_scale= y_df_prtp15,
                                                          inverse='yes',
                                                          for_inverse= ssp_base_pred_prtp15[i]))
    ssp_base_pred_inv_prtp1.append(scaling_transform(choose_scaler= scaler,
                                                         for_scale= y_df_prtp1,
                                                         inverse='yes',
                                                         for_inverse= ssp_base_pred_prtp1[i]))
    ssp_base_pred_inv_prtp3.append(scaling_transform(choose_scaler= scaler,
                                                         for_scale= y_df_prtp3,
                                                         inverse='yes',
                                                         for_inverse= ssp_base_pred_prtp3[i]))


In [ ]:
ssp_ccs_pred_inv_prtp15= []
ssp_ccs_pred_inv_prtp1= []
ssp_ccs_pred_inv_prtp3= []

for i in range(len(ssp_ccs_input_scld)):
    ssp_ccs_pred_inv_prtp15.append(scaling_transform(choose_scaler= scaler,
                                                          for_scale= y_df_prtp15,
                                                          inverse='yes',
                                                          for_inverse= ssp_ccs_pred_prtp15[i]))
    ssp_ccs_pred_inv_prtp1.append(scaling_transform(choose_scaler= scaler,
                                                         for_scale= y_df_prtp1,
                                                         inverse='yes',
                                                         for_inverse= ssp_ccs_pred_prtp1[i]))
    ssp_ccs_pred_inv_prtp3.append(scaling_transform(choose_scaler= scaler,
                                                         for_scale= y_df_prtp3,
                                                         inverse='yes',
                                                         for_inverse= ssp_ccs_pred_prtp3[i]))


### 5. Save prediction

In [ ]:
import os
''' making new directory folder '''
def making_dir(dir_name):
    # https://thispointer.com/how-to-create-a-directory-in-python/
    # Create target Directory if don't exist
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        print("Directory " , dir_name ,  " Created ")
    else:    
        print("Directory " , dir_name ,  " already exists")

    return

In [ ]:
pred_main_folder= "03_SCC_pred_folder_1030_iams_sspbase_ccs"
making_dir(dir_name= pred_main_folder)

pred_result_folder_fund_prtp1= pred_main_folder + "/fund/prtp1"
pred_result_folder_fund_prtp15= pred_main_folder + "/fund/prtp15"
pred_result_folder_fund_prtp3= pred_main_folder + "/fund/prtp3"

making_dir(dir_name= pred_result_folder_fund_prtp1)
making_dir(dir_name= pred_result_folder_fund_prtp15)
making_dir(dir_name= pred_result_folder_fund_prtp3)



pred_result_folder_page_prtp1= pred_main_folder + "/page/prtp1"
pred_result_folder_page_prtp15= pred_main_folder + "/page/prtp15"
pred_result_folder_page_prtp3= pred_main_folder + "/page/prtp3"

making_dir(dir_name= pred_result_folder_page_prtp1)
making_dir(dir_name= pred_result_folder_page_prtp15)
making_dir(dir_name= pred_result_folder_page_prtp3)



pred_result_folder_dice_prtp1= pred_main_folder + "/dice/prtp1"
pred_result_folder_dice_prtp15= pred_main_folder + "/dice/prtp15"
pred_result_folder_dice_prtp3= pred_main_folder + "/dice/prtp3"

making_dir(dir_name= pred_result_folder_dice_prtp1)
making_dir(dir_name= pred_result_folder_dice_prtp15)
making_dir(dir_name= pred_result_folder_dice_prtp3)



pred_result_folder_ssp_base_prtp1= pred_main_folder + "/ssp_base/prtp1"
pred_result_folder_ssp_base_prtp15= pred_main_folder + "/ssp_base/prtp15"
pred_result_folder_ssp_base_prtp3= pred_main_folder + "/ssp_base/prtp3"

making_dir(dir_name= pred_result_folder_ssp_base_prtp1)
making_dir(dir_name= pred_result_folder_ssp_base_prtp15)
making_dir(dir_name= pred_result_folder_ssp_base_prtp3)


pred_result_folder_ssp_ccs_prtp1= pred_main_folder + "/ssp_ccs/prtp1"
pred_result_folder_ssp_ccs_prtp15= pred_main_folder + "/ssp_ccs/prtp15"
pred_result_folder_ssp_ccs_prtp3= pred_main_folder + "/ssp_ccs/prtp3"

making_dir(dir_name= pred_result_folder_ssp_ccs_prtp1)
making_dir(dir_name= pred_result_folder_ssp_ccs_prtp15)
making_dir(dir_name= pred_result_folder_ssp_ccs_prtp3)

In [ ]:
fund_pred_inv_prtp15_df= []
fund_pred_inv_prtp1_df= []
fund_pred_inv_prtp3_df= []


for i in range(len(fund_pred_inv_prtp1)):
    fund_pred_inv_prtp15_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_1": fund_pred_inv_prtp15[i][:,0],
                                                 "scc_cs_2": fund_pred_inv_prtp15[i][:,1],
                                                 "scc_cs_3": fund_pred_inv_prtp15[i][:,2],
                                                 "scc_cs_4": fund_pred_inv_prtp15[i][:,3],}))

    fund_pred_inv_prtp1_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_5": fund_pred_inv_prtp1[i][:,0],
                                                 "scc_cs_6": fund_pred_inv_prtp1[i][:,1],
                                                 "scc_cs_7": fund_pred_inv_prtp1[i][:,2],
                                                 "scc_cs_8": fund_pred_inv_prtp1[i][:,3],}))

    fund_pred_inv_prtp3_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_9": fund_pred_inv_prtp3[i][:,0],
                                                 "scc_cs_10": fund_pred_inv_prtp3[i][:,1],
                                                 "scc_cs_11": fund_pred_inv_prtp3[i][:,2],
                                                 "scc_cs_12": fund_pred_inv_prtp3[i][:,3],}))

In [ ]:
page_pred_inv_prtp15_df= []
page_pred_inv_prtp1_df= []
page_pred_inv_prtp3_df= []


for i in range(len(page_pred_inv_prtp1)):
    page_pred_inv_prtp15_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_1": page_pred_inv_prtp15[i][:,0],
                                                 "scc_cs_2": page_pred_inv_prtp15[i][:,1],
                                                 "scc_cs_3": page_pred_inv_prtp15[i][:,2],
                                                 "scc_cs_4": page_pred_inv_prtp15[i][:,3],}))

    page_pred_inv_prtp1_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_5": page_pred_inv_prtp1[i][:,0],
                                                 "scc_cs_6": page_pred_inv_prtp1[i][:,1],
                                                 "scc_cs_7": page_pred_inv_prtp1[i][:,2],
                                                 "scc_cs_8": page_pred_inv_prtp1[i][:,3],}))

    page_pred_inv_prtp3_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_9": page_pred_inv_prtp3[i][:,0],
                                                 "scc_cs_10": page_pred_inv_prtp3[i][:,1],
                                                 "scc_cs_11": page_pred_inv_prtp3[i][:,2],
                                                 "scc_cs_12": page_pred_inv_prtp3[i][:,3],}))

In [ ]:
dice_pred_inv_prtp15_df= []
dice_pred_inv_prtp1_df= []
dice_pred_inv_prtp3_df= []


for i in range(len(dice_pred_inv_prtp1)):
    dice_pred_inv_prtp15_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_1": dice_pred_inv_prtp15[i][:,0],
                                                 "scc_cs_2": dice_pred_inv_prtp15[i][:,1],
                                                 "scc_cs_3": dice_pred_inv_prtp15[i][:,2],
                                                 "scc_cs_4": dice_pred_inv_prtp15[i][:,3],}))

    dice_pred_inv_prtp1_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_5": dice_pred_inv_prtp1[i][:,0],
                                                 "scc_cs_6": dice_pred_inv_prtp1[i][:,1],
                                                 "scc_cs_7": dice_pred_inv_prtp1[i][:,2],
                                                 "scc_cs_8": dice_pred_inv_prtp1[i][:,3],}))

    dice_pred_inv_prtp3_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_9": dice_pred_inv_prtp3[i][:,0],
                                                 "scc_cs_10": dice_pred_inv_prtp3[i][:,1],
                                                 "scc_cs_11": dice_pred_inv_prtp3[i][:,2],
                                                 "scc_cs_12": dice_pred_inv_prtp3[i][:,3],}))

In [ ]:
ssp_base_pred_inv_prtp15_df= []
ssp_base_pred_inv_prtp1_df= []
ssp_base_pred_inv_prtp3_df= []


for i in range(len(ssp_base_pred_inv_prtp1)):
    ssp_base_pred_inv_prtp15_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_1": ssp_base_pred_inv_prtp15[i][:,0],
                                                 "scc_cs_2": ssp_base_pred_inv_prtp15[i][:,1],
                                                 "scc_cs_3": ssp_base_pred_inv_prtp15[i][:,2],
                                                 "scc_cs_4": ssp_base_pred_inv_prtp15[i][:,3],}))

    ssp_base_pred_inv_prtp1_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_5": ssp_base_pred_inv_prtp1[i][:,0],
                                                 "scc_cs_6": ssp_base_pred_inv_prtp1[i][:,1],
                                                 "scc_cs_7": ssp_base_pred_inv_prtp1[i][:,2],
                                                 "scc_cs_8": ssp_base_pred_inv_prtp1[i][:,3],}))

    ssp_base_pred_inv_prtp3_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_9": ssp_base_pred_inv_prtp3[i][:,0],
                                                 "scc_cs_10": ssp_base_pred_inv_prtp3[i][:,1],
                                                 "scc_cs_11": ssp_base_pred_inv_prtp3[i][:,2],
                                                 "scc_cs_12": ssp_base_pred_inv_prtp3[i][:,3],}))

In [ ]:
ssp_ccs_pred_inv_prtp15_df= []
ssp_ccs_pred_inv_prtp1_df= []
ssp_ccs_pred_inv_prtp3_df= []


for i in range(len(ssp_ccs_pred_inv_prtp1)):
    ssp_ccs_pred_inv_prtp15_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_1": ssp_ccs_pred_inv_prtp15[i][:,0],
                                                 "scc_cs_2": ssp_ccs_pred_inv_prtp15[i][:,1],
                                                 "scc_cs_3": ssp_ccs_pred_inv_prtp15[i][:,2],
                                                 "scc_cs_4": ssp_ccs_pred_inv_prtp15[i][:,3],}))

    ssp_ccs_pred_inv_prtp1_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_5": ssp_ccs_pred_inv_prtp1[i][:,0],
                                                 "scc_cs_6": ssp_ccs_pred_inv_prtp1[i][:,1],
                                                 "scc_cs_7": ssp_ccs_pred_inv_prtp1[i][:,2],
                                                 "scc_cs_8": ssp_ccs_pred_inv_prtp1[i][:,3],}))

    ssp_ccs_pred_inv_prtp3_df.append(pd.DataFrame({"Year": year_all,
                                                 "scc_cs_9": ssp_ccs_pred_inv_prtp3[i][:,0],
                                                 "scc_cs_10": ssp_ccs_pred_inv_prtp3[i][:,1],
                                                 "scc_cs_11": ssp_ccs_pred_inv_prtp3[i][:,2],
                                                 "scc_cs_12": ssp_ccs_pred_inv_prtp3[i][:,3],}))

In [ ]:
page_pred_inv_prtp15_df[0], page_pred_inv_prtp1_df[1]

In [ ]:
for i in range(len(fund_pred_inv_prtp15_df)):
    fund_pred_inv_prtp15_df[i].to_csv(pred_result_folder_fund_prtp15 + "/" + "scc_result_prpt15_fund_mcs_" + str(i) + ".csv")
    fund_pred_inv_prtp1_df[i].to_csv(pred_result_folder_fund_prtp1 + "/" + "scc_result_prpt1_fund_mcs_" + str(i) + ".csv")
    fund_pred_inv_prtp3_df[i].to_csv(pred_result_folder_fund_prtp3 + "/" + "scc_result_prpt3_fund_mcs_" + str(i) + ".csv")

In [ ]:
for i in range(len(page_pred_inv_prtp15_df)):
    page_pred_inv_prtp15_df[i].to_csv(pred_result_folder_page_prtp15 + "/" + "scc_result_prpt15_page_mcs_" + str(i) + ".csv")
    page_pred_inv_prtp1_df[i].to_csv(pred_result_folder_page_prtp1 + "/" + "scc_result_prpt1_page_mcs_" + str(i) + ".csv")
    page_pred_inv_prtp3_df[i].to_csv(pred_result_folder_page_prtp3 + "/" + "scc_result_prpt3_page_mcs_" + str(i) + ".csv")

In [ ]:
for i in range(len(dice_pred_inv_prtp15_df)):
    dice_pred_inv_prtp15_df[i].to_csv(pred_result_folder_dice_prtp15 + "/" + "scc_result_prpt15_dice_mcs_" + str(i) + ".csv")
    dice_pred_inv_prtp1_df[i].to_csv(pred_result_folder_dice_prtp1 + "/" + "scc_result_prpt1_dice_mcs_" + str(i) + ".csv")
    dice_pred_inv_prtp3_df[i].to_csv(pred_result_folder_dice_prtp3 + "/" + "scc_result_prpt3_dice_mcs_" + str(i) + ".csv")

In [ ]:
for i in range(len(ssp_base_pred_inv_prtp15_df)):
    ssp_base_pred_inv_prtp15_df[i].to_csv(pred_result_folder_ssp_base_prtp15 + "/" + "scc_result_prpt15_ssp_base_mcs_" + str(i) + ".csv")
    ssp_base_pred_inv_prtp1_df[i].to_csv(pred_result_folder_ssp_base_prtp1 + "/" + "scc_result_prpt1_ssp_base_mcs_" + str(i) + ".csv")
    ssp_base_pred_inv_prtp3_df[i].to_csv(pred_result_folder_ssp_base_prtp3 + "/" + "scc_result_prpt3_ssp_base_mcs_" + str(i) + ".csv")

In [ ]:
for i in range(len(ssp_ccs_pred_inv_prtp15_df)):
    ssp_ccs_pred_inv_prtp15_df[i].to_csv(pred_result_folder_ssp_ccs_prtp15 + "/" + "scc_result_prpt15_ssp_ccs_mcs_" + str(i) + ".csv")
    ssp_ccs_pred_inv_prtp1_df[i].to_csv(pred_result_folder_ssp_ccs_prtp1 + "/" + "scc_result_prpt1_ssp_ccs_mcs_" + str(i) + ".csv")
    ssp_ccs_pred_inv_prtp3_df[i].to_csv(pred_result_folder_ssp_ccs_prtp3 + "/" + "scc_result_prpt3_ssp_ccs_mcs_" + str(i) + ".csv")